In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt

/var/folders/p6/zn1thsjj39x22wz5h1vhtxlh0000gn/T/ipykernel_1949/600834044.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [12]:
df=pd.read_csv('/Users/jeong-yula/Downloads/open/train.csv')

In [13]:
plt.rcParams['font.family'] = 'AppleGothic'

In [14]:
def year_encoder(x):
    output = x.replace('years', '')
    output = output.replace('year', '')
    output = output.replace('+', '')
    output = output.replace('<', '')
    output = output.replace('Unknown', '5').strip()
    output = int(output)

    return output

df['근로기간'] = df['근로기간'].apply(year_encoder)

In [15]:
drop_index = df[df['주택소유상태'] == 'ANY'].index

df = df.drop(drop_index)

In [16]:
def class_encoder(x):
    if x=='A':
        return 6
    elif x=='B':
        return 5
    elif x=='C':
        return 4
    elif x=='D':
        return 3
    elif x=='E':
        return 2
    elif x=='F':
        return 1
    else:
        return 0

df['대출등급']=df['대출등급'].apply(class_encoder)

In [17]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
le= LabelEncoder()
oe=OneHotEncoder()


In [18]:
oe.fit(df[['주택소유상태']])

OneHotEncoder()

In [19]:
oe.transform(df[['주택소유상태']])

<96293x3 sparse matrix of type '<class 'numpy.float64'>'
	with 96293 stored elements in Compressed Sparse Row format>

In [20]:
house_csr=oe.transform(df[['주택소유상태']])
house_csr_df=pd.DataFrame(house_csr.toarray(), columns=oe.get_feature_names_out())
house_csr_df.head()

,주택소유상태_MORTGAGE,주택소유상태_OWN,주택소유상태_RENT
0,0.0,0.0,1.0
1,1.0,0.0,0.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [21]:
df=pd.concat([df,house_csr_df],axis=1)

In [22]:
oe1=OneHotEncoder()

In [23]:
oe1.fit(df[['대출기간']])

OneHotEncoder()

In [24]:
period_csr=oe1.transform(df[['대출기간']])
period_csr_df=pd.DataFrame(period_csr.toarray(), columns=oe1.get_feature_names_out())
period_csr_df.head()

,대출기간_ 36 months,대출기간_ 60 months,대출기간_nan
0,1.0,0.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,1.0,0.0


In [25]:
df=pd.concat([df,period_csr_df],axis=1)

In [26]:
oe2=OneHotEncoder()

In [27]:
oe2.fit(df[['대출목적']])

OneHotEncoder()

In [28]:
oe2.transform(df[['대출목적']])

<96294x13 sparse matrix of type '<class 'numpy.float64'>'
	with 96294 stored elements in Compressed Sparse Row format>

In [29]:
purpose_csr=oe2.transform(df[['대출목적']])
purpose_csr_df=pd.DataFrame(purpose_csr.toarray(), columns=oe2.get_feature_names_out())
purpose_csr_df.head()

,대출목적_기타,대출목적_부채 통합,대출목적_소규모 사업,대출목적_신용 카드,대출목적_의료,대출목적_이사,대출목적_자동차,대출목적_재생 에너지,대출목적_주요 구매,대출목적_주택,대출목적_주택 개선,대출목적_휴가,대출목적_nan
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [30]:
df=pd.concat([df,purpose_csr_df],axis=1)

In [31]:


numeric_cols = ['연간소득', '총계좌수', '최근_2년간_연체_횟수', '총상환원금', '총상환이자', '총연체금액','부채_대비_소득_비율']

# 표준편차 사용하여 이상치 제거
def remove_outliers(df, col, threshold=3):
    mean = np.mean(df[col])
    std_dev = np.std(df[col])
    lower_bound = mean - threshold * std_dev
    upper_bound = mean + threshold * std_dev
    df= df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df

# 이상치 제거 후 데이터프레임 생성
df = df.copy()  # 복사본 생성

for col in numeric_cols:
    df = remove_outliers(df, col, threshold=3)

In [32]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
sd_sc=StandardScaler()
mm_sc=MinMaxScaler()


In [33]:
mm_sc.fit(df[['대출금액','총상환원금','총상환이자']])
df[['대출금액','총상환원금','총상환이자']]=mm_sc.transform(df[['대출금액','총상환원금','총상환이자']])

In [34]:
sd_sc.fit(df[['총연체금액','최근_2년간_연체_횟수','연체계좌수']])
df[['총연체금액','최근_2년간_연체_횟수','연체계좌수']]=sd_sc.transform(df[['총연체금액','최근_2년간_연체_횟수','연체계좌수']])

In [39]:
df.head()

,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급,주택소유상태_MORTGAGE,주택소유상태_OWN,주택소유상태_RENT,대출기간_ 36 months,대출기간_ 60 months,대출기간_nan,대출목적_기타,대출목적_부채 통합,대출목적_소규모 사업,대출목적_신용 카드,대출목적_의료,대출목적_이사,대출목적_자동차,대출목적_재생 에너지,대출목적_주요 구매,대출목적_주택,대출목적_주택 개선,대출목적_휴가,대출목적_nan
0,TRAIN_00000,0.276471,36 months,6.0,RENT,72000000.0,18.90,15.0,부채 통합,-0.435839,0.000000,0.000000,-0.003322,-0.069216,4.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,TRAIN_00001,0.323529,60 months,10.0,MORTGAGE,130800000.0,22.33,21.0,주택 개선,-0.435839,0.096842,0.135737,-0.003322,-0.069216,5.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,TRAIN_00002,0.264706,36 months,5.0,MORTGAGE,96000000.0,8.60,14.0,부채 통합,-0.435839,0.240735,0.088116,-0.003322,-0.069216,6.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,TRAIN_00003,0.323529,36 months,8.0,MORTGAGE,132000000.0,15.09,15.0,부채 통합,-0.435839,0.084464,0.088791,-0.003322,-0.069216,4.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,TRAIN_00004,0.411765,60 months,5.0,RENT,71736000.0,25.39,19.0,주요 구매,-0.435839,0.059245,0.086383,-0.003322,-0.069216,5.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [38]:
pd.options.display.max_columns = 50

In [40]:
df1=df.copy()

In [54]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 90618 entries, 0 to 96293
Data columns (total 28 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   대출금액             90618 non-null  float64
 1   근로기간             90618 non-null  float64
 2   연간소득             90618 non-null  float64
 3   부채_대비_소득_비율      90618 non-null  float64
 4   총계좌수             90618 non-null  float64
 5   최근_2년간_연체_횟수     90618 non-null  float64
 6   총상환원금            90618 non-null  float64
 7   총상환이자            90618 non-null  float64
 8   총연체금액            90618 non-null  float64
 9   연체계좌수            90618 non-null  float64
 10  대출등급             90618 non-null  float64
 11  주택소유상태_MORTGAGE  90617 non-null  float64
 12  주택소유상태_OWN       90617 non-null  float64
 13  주택소유상태_RENT      90617 non-null  float64
 14  대출기간_ 36 months  90618 non-null  float64
 15  대출기간_ 60 months  90618 non-null  float64
 16  대출목적_기타          90618 non-null  float64
 17  대출목적_부채 통합       

In [59]:
df1.isnull().sum()

대출금액               0
근로기간               0
연간소득               0
부채_대비_소득_비율        0
총계좌수               0
최근_2년간_연체_횟수       0
총상환원금              0
총상환이자              0
총연체금액              0
연체계좌수              0
대출등급               0
주택소유상태_MORTGAGE    1
주택소유상태_OWN         1
주택소유상태_RENT        1
대출기간_ 36 months    0
대출기간_ 60 months    0
대출목적_기타            0
대출목적_부채 통합         0
대출목적_소규모 사업        0
대출목적_신용 카드         0
대출목적_의료            0
대출목적_이사            0
대출목적_자동차           0
대출목적_재생 에너지        0
대출목적_주요 구매         0
대출목적_주택            0
대출목적_주택 개선         0
대출목적_휴가            0
dtype: int64

In [63]:
df1.isnull()

,대출금액,근로기간,연간소득,부채_대비_소득_비율,총계좌수,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급,주택소유상태_MORTGAGE,주택소유상태_OWN,주택소유상태_RENT,대출기간_ 36 months,대출기간_ 60 months,대출목적_기타,대출목적_부채 통합,대출목적_소규모 사업,대출목적_신용 카드,대출목적_의료,대출목적_이사,대출목적_자동차,대출목적_재생 에너지,대출목적_주요 구매,대출목적_주택,대출목적_주택 개선,대출목적_휴가
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96287,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
96289,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
96290,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
96291,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [62]:
df1=df1.dropna(axis=0)

In [53]:
df1.head()

,대출금액,근로기간,연간소득,부채_대비_소득_비율,총계좌수,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급,주택소유상태_MORTGAGE,주택소유상태_OWN,주택소유상태_RENT,대출기간_ 36 months,대출기간_ 60 months,대출목적_기타,대출목적_부채 통합,대출목적_소규모 사업,대출목적_신용 카드,대출목적_의료,대출목적_이사,대출목적_자동차,대출목적_재생 에너지,대출목적_주요 구매,대출목적_주택,대출목적_주택 개선,대출목적_휴가
0,0.276471,6.0,72000000.0,18.90,15.0,-0.435839,0.000000,0.000000,-0.003322,-0.069216,4.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.323529,10.0,130800000.0,22.33,21.0,-0.435839,0.096842,0.135737,-0.003322,-0.069216,5.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.264706,5.0,96000000.0,8.60,14.0,-0.435839,0.240735,0.088116,-0.003322,-0.069216,6.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.323529,8.0,132000000.0,15.09,15.0,-0.435839,0.084464,0.088791,-0.003322,-0.069216,4.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.411765,5.0,71736000.0,25.39,19.0,-0.435839,0.059245,0.086383,-0.003322,-0.069216,5.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [42]:
df1=df1.drop(['ID'],axis=1)

In [44]:
df1=df1.drop(['대출기간'],axis=1)

In [46]:
df1=df1.drop(['주택소유상태'],axis=1)

In [48]:
df1=df1.drop(['대출목적'],axis=1)

In [50]:
df1=df1.drop(['대출기간_nan'],axis=1)

In [52]:
df1=df1.drop(['대출목적_nan'],axis=1)

In [73]:
from sklearn.model_selection import train_test_split

X =df1.drop(['대출등급'],axis=1)
y = df1[['대출등급']]

train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 42) # 학습데이터와 평가데이터의 비율을 8:2 로 분할| 
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape) # 데이터 개수 확인

(72493, 27) (18124, 27) (72493, 1) (18124, 1)


In [79]:
#기본적인 randomforest모형

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score# 정확도 함수
from sklearn.metrics import f1_score

clf = RandomForestClassifier(n_estimators=20, max_depth=5,random_state=0)
clf.fit(train_x,train_y)

predict1 = clf.predict(test_x)
print(accuracy_score(test_y,predict1))



/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/base.py:1351: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


0.42308541160891633


In [75]:
# sample 100개, tree depth - 20

clf = RandomForestClassifier(n_estimators=100, max_depth=20,random_state=0)
clf.fit(train_x,train_y)

predict2 = clf.predict(test_x)
print(accuracy_score(test_y,predict2))

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/base.py:1351: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


0.707459721915692


In [76]:
# sample 100개, tree depth - 100(max)

clf = RandomForestClassifier(n_estimators=100, max_depth=100,random_state=0)
clf.fit(train_x,train_y)

predict2 = clf.predict(test_x)
print(accuracy_score(test_y,predict2))

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/base.py:1351: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


0.7247296402560142
